In [15]:
import os
import numpy as np
import pandas as pd
import random
    
from glob import glob

from scipy.io import wavfile
from scipy.signal import stft

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical

class DatasetGenerator():
    def __init__(self, label_set, 
                 sample_rate=16000):
        
        self.label_set = label_set
        self.sample_rate = sample_rate
            
    # Covert string to numerical classes              
    def text_to_labels(self, text):
        return self.label_set.index(text)
    
    # Reverse translation of numerical classes back to characters
    def labels_to_text(self, labels):
        return self.label_set[labels]               
        
    def load_data(self, DIR):

        # Get all paths inside DIR that ends with wav
        wav_files = glob(os.path.join(DIR, '*/*wav'))
        wav_files = [x.split(sep='\\')[1] + '/' + x.split(sep='\\')[2] for x in wav_files]
        
        # Loop over files to get samples
        data = []
        for e in wav_files:
            
            label, name = e.split('/')
            if label in self.label_set:
                label_id = self.text_to_labels(label)
                fle = os.path.join(DIR, e)
                
                sample = (label, label_id, name, fle)
                data.append(sample)
            
        # Data Frames with samples' labels and paths     
        df = pd.DataFrame(data, columns = ['label', 'label_id', 'user_id', 'wav_file'])
        
        self.df = df
        
        return self.df

    def apply_train_test_split(self, test_size, random_state):
        
        self.df_train, self.df_test = train_test_split(self.df, 
                                                       test_size=test_size,
                                                       random_state=random_state)
        
    def apply_train_val_split(self, val_size, random_state):
        
        self.df_train, self.df_val = train_test_split(self.df_train, 
                                                      test_size=val_size, 
                                                      random_state=random_state)
        
    def read_wav_file(self, x):
        # Read wavfile using scipy wavfile.read
        _, wav = wavfile.read(x) 
        # Normalize
        wav = wav.astype(np.float32) / np.iinfo(np.int16).max
            
        return wav
    
    def process_wav_file(self, x, threshold_freq=5500, eps=1e-10):
        # Read wav file to array
        wav = self.read_wav_file(x)
        # Sample rate
        L = self.sample_rate
        # If longer then randomly truncate
        if len(wav) > L:
            i = np.random.randint(0, len(wav) - L)
            wav = wav[i:(i+L)]  
        # If shorter then randomly add silence
        elif len(wav) < L:
            rem_len = L - len(wav)
            silence_part = np.random.randint(-100,100,16000).astype(np.float32) / np.iinfo(np.int16).max
            j = np.random.randint(0, rem_len)
            silence_part_left  = silence_part[0:j]
            silence_part_right = silence_part[j:rem_len]
            wav = np.concatenate([silence_part_left, wav, silence_part_right])
        # Create spectrogram using discrete FFT (change basis to frequencies)
        freqs, times, spec = stft(wav, L, nperseg = 400, noverlap = 240, nfft = 512, padded = False, boundary = None)
        # Cut high frequencies
        if threshold_freq is not None:
            spec = spec[freqs <= threshold_freq,:]
            freqs = freqs[freqs <= threshold_freq]
        # Log spectrogram
        amp = np.log(np.abs(spec)+eps)
    
        return np.expand_dims(amp, axis=2) 

    def generator(self, batch_size, mode):
        #print(type(self)) -> class daatset generator
        while True:
            # Depending on mode select DataFrame with paths
            if mode == 'train':
                df = self.df_train 
                ids = random.sample(range(df.shape[0]), df.shape[0])
            elif mode == 'val':
                df = self.df_val
                ids = list(range(df.shape[0]))
            elif mode == 'test':
                df = self.df_test
                ids = list(range(df.shape[0]))
                #print(ids)
            else:
                raise ValueError('The mode should be either train, val or test.')
                
            # Create batches (for training data the batches are randomly permuted)
            for start in range(0, len(ids), batch_size):
                X_batch = []
                if mode != 'test': 
                    y_batch = []
                end = min(start + batch_size, len(ids))
                i_batch = ids[start:end]
                for i in i_batch:
                    X_batch.append(self.process_wav_file(df.wav_file.values[i]))
                    if mode != 'test':
                        y_batch.append(df.label_id.values[i])
                X_batch = np.array(X_batch)

                if mode != 'test':
                    y_batch = to_categorical(y_batch, num_classes = len(self.label_set))
                    yield (X_batch, y_batch)
                else:
                    yield X_batch


In [16]:
import numpy as np
from scipy.io import wavfile
#import matplotlib.pyplot as plt
 
DIR = 'D:/college/TRF/LEVEL-II/Task_3'
'''fns = ['/bed/00f0204f_nohash_0.wav',
       '/cat/00b01445_nohash_0.wav',
       '/happy/0a2b400e_nohash_0.wav']'''
SAMPLE_RATE = 16000
 
def read_wav_file(x):
    # Read wavfile using scipy wavfile.read
    _, wav = wavfile.read(x) 
    # Normalize
    wav = wav.astype(np.float32) / np.iinfo(np.int16).max
        
    return wav
 
'''fig = plt.figure(figsize=(14, 8))
for i, fn in enumerate(fns):
    wav = read_wav_file(DIR + fn)
 
    ax = fig.add_subplot(3,1,i+1)
    ax.set_title('Raw wave of ' + fn)
    ax.set_ylabel('Amplitude')
    ax.plot(np.linspace(0, SAMPLE_RATE/len(wav), SAMPLE_RATE), wav)
fig.tight_layout()
'''

from scipy.signal import stft
 
def log_spectrogram(wav):
    freqs, times, spec = stft(wav, SAMPLE_RATE, nperseg = 400, noverlap = 240, nfft = 512, 
                              padded = False, boundary = None)
    # Log spectrogram
    amp = np.log(np.abs(spec)+1e-10)
    
    return freqs, times, amp
 
'''fig = plt.figure(figsize=(14, 8))
for i, fn in enumerate(fns):
    wav = read_wav_file(DIR + fn)
    freqs, times, amp = log_spectrogram(wav)
    
    ax = fig.add_subplot(3,1,i+1)
    ax.imshow(amp, aspect='auto', origin='lower', 
               extent=[times.min(), times.max(), freqs.min(), freqs.max()])
    ax.set_title('Spectrogram of ' + fn)
    ax.set_ylabel('Freqs in Hz')
    ax.set_xlabel('Seconds')
fig.tight_layout()
'''

"fig = plt.figure(figsize=(14, 8))\nfor i, fn in enumerate(fns):\n    wav = read_wav_file(DIR + fn)\n    freqs, times, amp = log_spectrogram(wav)\n    \n    ax = fig.add_subplot(3,1,i+1)\n    ax.imshow(amp, aspect='auto', origin='lower', \n               extent=[times.min(), times.max(), freqs.min(), freqs.max()])\n    ax.set_title('Spectrogram of ' + fn)\n    ax.set_ylabel('Freqs in Hz')\n    ax.set_xlabel('Seconds')\nfig.tight_layout()\n"

In [25]:

from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping
 
#from dataset import DatasetGenerator
 
DIR = 'D:/college/TRF/LEVEL-II/Task_3'
 
INPUT_SHAPE = (177,98,1)
BATCH = 1
EPOCHS = 15
 
LABELS = 'bed bird on one'.split()
NUM_CLASSES = len(LABELS)


dsGen = DatasetGenerator(label_set=LABELS) 
# Load DataFrame with paths/labels 
df = dsGen.load_data(DIR)


dsGen.apply_train_test_split(test_size=0.1, random_state=2018)
dsGen.apply_train_val_split(val_size=0.1, random_state=2018)
print(len(dsGen.df_test))


9


In [26]:

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten
                          
def deep(features_shape, num_classes, act='relu'):

    # Input
    x = Input(name='inputs', shape=features_shape, dtype='float32')
    o = x
    
    # Flatten
    o = Flatten(name='flatten')(o)
    
    # Dense layer
    o = Dense(512, activation=act, name='dense1')(o)
    o = Dense(512, activation=act, name='dense2')(o)
    o = Dense(512, activation=act, name='dense3')(o)
    
    # Predictions
    o = Dense(num_classes, activation='softmax', name='pred')(o)
    
    # Print network summary
    Model(inputs=x, outputs=o).summary()
    
    return Model(inputs=x, outputs=o)

from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
 
def deep_cnn(features_shape, num_classes, act='relu'):
 
    x = Input(name='inputs', shape=features_shape, dtype='float32')
    o = x
    
    # Block 1
    o = Conv2D(32, (3, 3), activation=act, padding='same', strides=1, name='block1_conv', input_shape=features_shape)(o)
    o = MaxPooling2D((3, 3), strides=(2,2), padding='same', name='block1_pool')(o)
    o = BatchNormalization(name='block1_norm')(o)
    
    # Block 2
    o = Conv2D(32, (3, 3), activation=act, padding='same', strides=1, name='block2_conv')(o)
    o = MaxPooling2D((3, 3), strides=(2,2), padding='same', name='block2_pool')(o)
    o = BatchNormalization(name='block2_norm')(o)
 
    # Block 3
    o = Conv2D(32, (3, 3), activation=act, padding='same', strides=1, name='block3_conv')(o)
    o = MaxPooling2D((3, 3), strides=(2,2), padding='same', name='block3_pool')(o)
    o = BatchNormalization(name='block3_norm')(o)
 
    # Flatten
    o = Flatten(name='flatten')(o)
    
    # Dense layer
    o = Dense(64, activation=act, name='dense')(o)
    o = BatchNormalization(name='dense_norm')(o)
    o = Dropout(0.2, name='dropout')(o)
    
    # Predictions
    o = Dense(num_classes, activation='softmax', name='pred')(o)
 
    # Print network summary
    Model(inputs=x, outputs=o).summary()
    
    return Model(inputs=x, outputs=o)


model = deep_cnn(INPUT_SHAPE, NUM_CLASSES)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 177, 98, 1)        0         
_________________________________________________________________
block1_conv (Conv2D)         (None, 177, 98, 32)       320       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 89, 49, 32)        0         
_________________________________________________________________
block1_norm (BatchNormalizat (None, 89, 49, 32)        128       
_________________________________________________________________
block2_conv (Conv2D)         (None, 89, 49, 32)        9248      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 45, 25, 32)        0         
_________________________________________________________________
block2_norm (BatchNormalizat (None, 45, 25, 32)        128       
__________

In [27]:

callbacks = [EarlyStopping(monitor='val_acc', patience=4, verbose=1, mode='max')]
 
history = model.fit_generator(generator=dsGen.generator(BATCH, mode='train'),
                              steps_per_epoch=int(np.ceil(len(dsGen.df_train)/BATCH)),
                              epochs=EPOCHS,
                              verbose=1,
                              callbacks=callbacks,
                              validation_data=dsGen.generator(BATCH, mode='val'),
                              validation_steps=int(np.ceil(len(dsGen.df_val)/BATCH)))


Epoch 1/15
64/64 [==============================] - 7s 105ms/step - loss: 1.3917 - acc: 0.1406 - val_loss: 8.0607 - val_acc: 0.5000
Epoch 2/15
64/64 [==============================] - 2s 31ms/step - loss: 1.3898 - acc: 0.1719 - val_loss: 8.0659 - val_acc: 0.5000
Epoch 3/15
64/64 [==============================] - 2s 31ms/step - loss: 1.3891 - acc: 0.2188 - val_loss: 11.2065 - val_acc: 0.2500
Epoch 4/15
64/64 [==============================] - 2s 32ms/step - loss: 1.3874 - acc: 0.2500 - val_loss: 8.4868 - val_acc: 0.3750
Epoch 5/15
64/64 [==============================] - 2s 32ms/step - loss: 1.3881 - acc: 0.2188 - val_loss: 8.7107 - val_acc: 0.3750
Epoch 00005: early stopping


In [42]:

#Reading audio from user
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 1
RATE = 16000 #sample rate
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "D:/college/TRF/LEVEL-II/Task_3/on/new.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")

frames = []
#print(int(RATE / CHUNK * RECORD_SECONDS))

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")
#print(frames)
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()
#end of reading audio

from scipy.io.wavfile import read
threshold_freq=5500 
eps=1e-10
wav = (read(WAVE_OUTPUT_FILENAME))[1]

from pydub import AudioSegment
from pydub.playback import play 
audio = AudioSegment.from_wav(WAVE_OUTPUT_FILENAME)
play(audio)

from pydub import AudioSegment
from pydub.silence import split_on_silence

sound_file = AudioSegment.from_wav(WAVE_OUTPUT_FILENAME)
audio_chunks = split_on_silence(sound_file, 
    # must be silent for at least half a second
    min_silence_len=100,

    # consider it silent if quieter than -16 dBFS
    silence_thresh=-20
)
count = 0
for i, chunk in enumerate(audio_chunks):

    out_file = "D:/college/TRF/LEVEL-II/Task_3/chunk{0}.wav".format(i)
    print ("exporting", out_file)
    chunk.export(out_file, format="wav")
    count += 1

# Sample rate
L = 16000
print(len(wav))
# If longer then randomly truncate
if len(wav) > L:
    i = np.random.randint(0, len(wav) - L)
    wav = wav[i:(i+L)]  
    print(len(wav))
# If shorter then randomly add silence
elif len(wav) < L:
    rem_len = L - len(wav)
    silence_part = np.random.randint(-100,100,16000).astype(np.float32) / np.iinfo(np.int16).max
    j = np.random.randint(0, rem_len)
    silence_part_left  = silence_part[0:j]
    silence_part_right = silence_part[j:rem_len]
    print(len(silence_part_left)," ",len(wav)," ",len(silence_part_right))
    wav = np.concatenate([silence_part_left, wav, silence_part_right])
# Create spectrogram using discrete FFT (change basis to frequencies)
freqs, times, spec = stft(wav, L, nperseg = 400, noverlap = 240, nfft = 512, padded = False, boundary = None)
# Cut high frequencies
if threshold_freq is not None:
    spec = spec[freqs <= threshold_freq,:]
    freqs = freqs[freqs <= threshold_freq]
    

import pandas as pd
e = "on/new.wav"
    
label, name = e.split('/')
label_id = dsGen.text_to_labels(label)
for j in range(count):
    fle = "D:/college/TRF/LEVEL-II/Task_3/chunk{0}.wav".format(j)
    sample = (label, label_id, name, fle)
    n_df = pd.DataFrame(data = [sample],columns = ['label', 'label_id', 'user_id', 'wav_file'])
    dsGen.df_test = dsGen.df_test.append(n_df)

from keras.utils import to_categorical
import random


* recording
* done recording
exporting D:/college/TRF/LEVEL-II/Task_3/chunk0.wav
exporting D:/college/TRF/LEVEL-II/Task_3/chunk1.wav
47104
16000


In [43]:
dsGen.df_test = dsGen.df_test[-count:]

In [44]:
print(dsGen.df_test)

  label  label_id  user_id                                   wav_file
0    on         2  new.wav  D:/college/TRF/LEVEL-II/Task_3/chunk0.wav
0    on         2  new.wav  D:/college/TRF/LEVEL-II/Task_3/chunk1.wav


In [45]:
y_pred_proba = model.predict_generator(dsGen.generator(BATCH, mode='test'), 
                                     int(np.ceil(len(dsGen.df_test)/BATCH)), 
                                     verbose=1)

y_pred = np.argmax(y_pred_proba, axis=1)

y_true = dsGen.df_test['label_id'].values
print(y_pred)

2/2 [==============================] - 0s 18ms/step
[0 2]


In [46]:
ans = []
print("You Said: ")
for j in range(count):
    ans.append(dsGen.labels_to_text(y_pred[len(dsGen.df_test) - 1 - j]))
    print(ans[j],end = " ")


You Said: 
on bed 